# `High-level idea`

- Pull current weather for a city from OpenWeather.

- Load historical CSV.

- Train one classifier to predict RainTomorrow (Yes/No).

- Train two regressors to predict the next Temp and next Humidity from their current value (5-step ahead via recursive prediction).

- Use current readings + trained models to predict rain now and forecast next 5 hours of temp & humidity.

- Print everything.

### **Section 1: Import Libraries**

In [26]:
import requests  # this library helps us to fetchh data from API
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from datetime import datetime, timedelta

In [27]:
API_KEY = '5f34de09700764f449651c6b7585fd25'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

# **1.  get_current_weather(city)**

### `When called:` inside weather_view() right after you input the city.

### `Does:` Calls OpenWeather API and extracts the fields you’ll use.
### `Returns:` A Python dict like:

In [37]:
# this function fetch Weather data for given specific city

def get_current_weather(city):

  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir': data['wind']['deg'],
      'pressure': data['main']['pressure'],
      'Wind_Gust_Speed': data['wind']['speed']
  }

# **2.  read_historical_data(filename)**

### `When called:` in weather_view() after current weather is fetched.
### `Does:` Loads your CSV, drops missing rows and duplicates.
### `Returns:` A cleaned DataFrame df.


In [29]:
def read_historical_data(filename):

  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()

  return df

# **3.  prepare_data(data)**

### `When called:` in weather_view() to make inputs for the rain classifier.
### `Does:` Encodes categorical columns and selects features/target. Returns: (x, y, le)

x: features (DataFrame/array)

y: target labels (Series/array)

le: a LabelEncoder instance

In [30]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  # define the feature variable and target variables
  x = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity',
       'Pressure', 'Temp']]
  y = data['RainTomorrow']

  return x, y, le

# **4.  train_rain_model(x, y)**

### `When called:` in weather_view() after prepare_data.
### `Does:` Splits into train/test → fits a RandomForestClassifier.
### `Returns:` the trained classifier rain_model.

***model for classification data***

In [31]:
def train_rain_model(x, y):

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(x_train, y_train)

  y_pred = model.predict(x_test)
  print('Accuracy Score for Rain Model')
  print(accuracy_score(y_test, y_pred))

  return model

***model for regression data***

# **5.  prepare_regression_data(data, feature)**

### `When called:` twice in weather_view() to build data for:
Temperature model (feature='Temp')

Humidity model (feature='Humidity')
### `Does:` Creates pairs (current_value → next_value) from a single column.
### `Returns:` (x, y) as NumPy arrays:

x.shape = (n-1, 1) (current value)

y.shape = (n-1,) (next value)


In [32]:
def prepare_regression_data(data, feature): # feature is the column name which we use for regression

  x, y = [], []

  for i in range(len(data) - 1):

    #Loop through the rows.
    #    Take the current value (iloc[i]) of the feature as input.
    #    Take the next value (iloc[i+1]) as the output/label.

    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  x = np.array(x).reshape(-1, 1)
  y = np.array(y)
  return x, y

# 6. **train_regression_model(x, y)**

### When called: in weather_view() for temp model and humidity model.
### Does: Fits a RandomForestRegressor.
### Returns: the trained regressor.

### `So this step is only about learning patterns from historical data, Without this step, the model won’t know how to predict anything.`

In [33]:
def train_regression_model(x, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x, y)
  return model

# 7. **predict_future(model, current_value)**

### What it does:
- Uses the already trained model (from step 6).

- Starts from a current value (e.g., today’s temp).

- Predicts next value → then feeds it back as input → predicts again.

- Repeats this 5 times → gives a sequence of 5 future predictions.

### `This step is about using the trained model to forecast future values, not training.`

In [34]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])

  return predictions[1:]

In [35]:
import pytz

# **weather_view() ← Main driver (entry point)**

### 1. Get city from user
### 2. Fetch current weather:
### 3. Read historical data
### 4. Prepare & train the rain classifier
### 5. Convert current wind degrees → compass label → encoded number:

  - Compute wind_deg = current_weather['wind_gust_dir'] % 360

  - Search compass_points for the interval containing wind_deg

  - If that compass label is present in le.classes_, encode it; else set -1

### 6. Build a one-row DataFrame of the current conditions (feature names matching your training columns)
### 7. Predict rain (Yes/No) for now
### 8. Build & train regression models for Temp and Humidity
### 9. Forecast next 5 hours (recursive) -> using 'predict_future'
### 10. Make display times for the next 5 hours (Asia/Karachi), round to the next full hour.
### 11. Print current stats, rain prediction, then the 5 temperature and 5 humidity forecasts.
### 12. Run the whole pipeline because at the bottom you have:


In [39]:
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)

  # load historical data
  historical_data = read_historical_data('/content/weather.csv')

  # prepare and train the rain prediction model
  x, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(x, y)

  # The problem:
  #      You have wind direction in degrees (like 90°, 180°, 270°).
  #      But your dataset/model uses compass labels like "E", "S", "W", etc.
  #      And your machine learning model cannot understand "E", "W" as text — it needs numbers.

  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)]

  compass_direction = next(
      point for point, start, end in compass_points if start <= wind_deg < end
  )

  # Encode the compass direction if it exists in LabelEncoder classes
  if compass_direction in le.classes_:
      compass_direction_encoded = le.transform([compass_direction])[0]
  else:
      compass_direction_encoded = -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['Wind_Gust_Speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }

  current_df = pd.DataFrame([current_data])

  # rain prediction

  rain_prediction = rain_model.predict(current_df)[0]

  # prepare regression model for temperature and humidity

  x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(x_temp, y_temp)
  hum_model = train_regression_model(x_hum, y_hum)

  # predict future temperature and humidity

  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])

  # prepare time for future predictions

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  # Display results

  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}")
  print(f"Feels Like: {current_weather['feels_like']}")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Predictions: ")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print('\nFuture Humidity Predictions: ')

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")


weather_view()


Enter any city name: lucknow
Accuracy Score for Rain Model
0.8493150684931506
City: lucknow, IN
Current Temperature: 31
Feels Like: 38
Minimum Temperature: 31°C
Maximum Temperature: 31°C
Humidity: 79%
Weather Prediction: mist
Rain Prediction: Yes

Future Temperature Predictions: 
19:00: 31.6°C
20:00: 28.0°C
21:00: 27.8°C
22:00: 25.9°C
23:00: 28.9°C

Future Humidity Predictions: 
19:00: 54.0%
20:00: 58.1%
21:00: 54.2%
22:00: 58.1%
23:00: 54.2%
